In [1]:
import pandas as pd

In [2]:
import itertools

In [57]:
import numpy as np 
import math 


In [4]:
# data from https://github.com/potamides/uniformers/tree/main
# df_de = pd.read_json('dataset/QuaTrain-de.json', lines=True)
df_en = pd.read_json('dataset/QuaTrain-en.json', lines=True)

In [5]:
df_en.head()

,text,language,rhyme,meter,alliteration
0,"[Thou , dost overmuch respect, That , which wi...",en,AABB,iambus,0.088559
1,"[That , which will thy harme effect ;, But , s...",en,ABBC,iambus,0.089307
2,"[But , some other things there bee ,, Which wi...",en,AABB,iambus,0.062548
3,"[Which will more advantage thee :, Search thy ...",en,ABBC,iambus,0.066380
4,"[Search thy heart ; and , thou shalt , there ,...",en,AABB,iambus,0.053071


In [6]:
for l in list(df_en["text"])[:15]:
    print (l, "\n")

['Thou , dost overmuch respect', 'That , which will thy harme effect ;', 'But , some other things there bee ,', 'Which will more advantage thee :'] 

['That , which will thy harme effect ;', 'But , some other things there bee ,', 'Which will more advantage thee :', 'Search thy heart ; and , thou shalt , there ,'] 

['But , some other things there bee ,', 'Which will more advantage thee :', 'Search thy heart ; and , thou shalt , there ,', 'Soone discover , what they are :'] 

['Which will more advantage thee :', 'Search thy heart ; and , thou shalt , there ,', 'Soone discover , what they are :', 'Yea , thine Emblem showes thee , too ,'] 

['Search thy heart ; and , thou shalt , there ,', 'Soone discover , what they are :', 'Yea , thine Emblem showes thee , too ,', 'What to shunne ; and , what to doe .'] 

['It is a little fear ’ d , that you', 'Are to your owne Designes , untrue ;', 'And , that , if you more constant were ,', 'You would be richer , then you are ,'] 

['Are to your owne 

In [7]:
def get_unique_quatrains(df):
    last_t = ""
    overlapping = []
    unique_rows = []
    for i, row in df.iterrows():
        if list(row["text"])[0] == last_t:
            overlapping.append(row)
        elif len(overlapping) == 0:
            overlapping = [row]
        else:
            unique_rows.append(overlapping[0])
            overlapping = [row]
        last_t = list(row["text"])[1]
    unique_rows.append(overlapping[0]) 
    #print(unique_rows)   
    df_unique = pd.DataFrame(unique_rows)
    return(df_unique)

In [8]:
df_en_unique = get_unique_quatrains(df_en)

In [10]:
df_en_unique.head()

,text,language,rhyme,meter,alliteration
0,"[Thou , dost overmuch respect, That , which wi...",en,AABB,iambus,0.088559
5,"[It is a little fear ’ d , that you, Are to yo...",en,AABB,iambus,0.042092
10,"[How rich or poore soe ’ re thou be ,, Thou , ...",en,AABB,iambus,0.009518
15,"[Much Liberty , thou hast assum ’ d ;, And , h...",en,AABB,iambus,0.009096
20,"[Thou seekest Honour , to obtaine ,, By meanes...",en,AABB,iambus,0.032559


In [11]:
len(df_en)

662885

In [12]:
len(df_en_unique)

126307

In [13]:
def check_common_elements(lists):
    for i in range(len(lists)):
        for j in range(i + 1, len(lists)):
            if any(elem in lists[i] for elem in lists[j]):
                return True
    return False

# Test the function
lists = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [8]]
result = check_common_elements(lists)
if result:
    print("There are lists with common elements.")
else:
    print("No lists share any elements.")

There are lists with common elements.


In [14]:
def find_common_elements(lists):
    common_elements = []
    for i in range(len(lists)):
        for j in range(i + 1, len(lists)):
            common = [elem for elem in lists[i] if elem in lists[j]]
            if common:
                common_elements.append((lists[i], lists[j], i, j, common))
    return common_elements

# Test the function
lists = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [3, 10], [1, 7, 8, 3]]
common_lists = find_common_elements(lists)

if common_lists:
    print("The following lists share elements:")
    for common_list in common_lists:
        list1, list2, i, j, common_elements = common_list
        print(f"{list1} and {list2}: {common_elements}, index: {i} and {j}")
else:
    print("No lists share any elements.")

The following lists share elements:
[1, 2, 3] and [3, 10]: [3], index: 0 and 3
[1, 2, 3] and [1, 7, 8, 3]: [1, 3], index: 0 and 4
[7, 8, 9] and [1, 7, 8, 3]: [7, 8], index: 2 and 4
[3, 10] and [1, 7, 8, 3]: [3], index: 3 and 4


In [27]:
common_lists = find_common_elements(list(df_de_unique["text"]))
common_lists = sorted(common_lists, key=lambda k: len(k[4]), reverse=True)
if common_lists:
    print("The following lists share elements:")
    for common_list in common_lists:
        list1, list2, i, j, common_elements = common_list
        if not i in to_delete or j in to_delete:
            to_delete.append(i)
    else:
        print("No lists share any elements.")

print(to_delete)

KeyboardInterrupt: 

In [15]:
vocab, inv = np.unique(np.concatenate(list(df_en_unique["text"])), return_inverse=True)

In [16]:
print(inv)

[371438 315167  92886 ... 185203  34077 196028]


In [33]:
inv.size/4

126307.0

In [18]:
len(vocab)

472903

In [19]:
quads = np.resize(inv, (int(inv.size/4), 4))

In [20]:
print(quads)

[[371438 315167  92886 426566]
 [206210  73262  26180 454229]
 [171706 371431  25901  75214]
 ...
 [191785 173325 173449  65794]
 [ 99761 322440 384271 409696]
 [ 36617 185203  34077 196028]]


In [21]:
len(quads)

126307

In [22]:
def find_overlaps(arr):
    overlaps = []
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            overlap_indices = np.where(np.in1d(arr[i], arr[j]))[0]
            if len(overlap_indices) > 0:
                overlaps.append((i, j, overlap_indices))
    return overlaps

# Test the function
if __name__ == "__main__":
    arr = np.array([
        [1, 2, 3, 4],
        [3, 4, 5, 6],
        [7, 8, 9, 10],
        [1, 4, 7, 10]
    ])

    overlaps = find_overlaps(arr)
    for overlap in overlaps:
        print("Overlap between arrays {} and {}: indices {}".format(overlap[0], overlap[1], overlap[2]))

Overlap between arrays 0 and 1: indices [2 3]
Overlap between arrays 0 and 3: indices [0 3]
Overlap between arrays 1 and 3: indices [1]
Overlap between arrays 2 and 3: indices [0 3]


In [61]:
print(overlaps)

[(0, 1, array([2, 3], dtype=int64)), (0, 3, array([0, 3], dtype=int64)), (1, 3, array([1], dtype=int64)), (2, 3, array([0, 3], dtype=int64))]


In [62]:
overlaps_ordered = sorted(overlaps, key=lambda k: len(k[2]), reverse=True)

print(overlaps_ordered)

[(0, 1, array([2, 3], dtype=int64)), (0, 3, array([0, 3], dtype=int64)), (2, 3, array([0, 3], dtype=int64)), (1, 3, array([1], dtype=int64))]


In [64]:
to_delete

[0, 2, 1]

In [66]:
overlaps_de = find_overlaps(quads[:1000])

In [67]:
overlaps_ordered = sorted(overlaps_de, key=lambda k: len(k[2]), reverse=True)
to_delete = []
for i, j, o in overlaps_ordered:
    if not i in to_delete or j in to_delete:
        to_delete.append(i)

In [68]:
print(to_delete)

[167, 713, 774, 922, 186, 187, 188, 189, 190, 191, 192, 316, 412]


In [71]:
def find_overlaps(arr):
    # Flatten the input array to work with 1D arrays
    flat_arr = np.vstack(arr)
    
    # Create an array of unique elements and their counts
    unique_elements, counts = np.unique(flat_arr, return_counts=True)
    
    # Find elements that appear in more than one array
    overlapping_elements = unique_elements[counts > 1]
    
    # Initialize a list to store the overlaps
    overlaps = []
    
    # Iterate through each overlapping element
    for element in overlapping_elements:
        # Find the indices of arrays that contain the element
        array_indices = [i for i, a in enumerate(arr) if np.isin(element, a)]
        print("array indices: ", array_indices)
        # Append the indices to the overlaps list
        overlaps.extend((i, j, element) for i in array_indices for j in array_indices if i < j)
    
    return overlaps

# Test the function
if __name__ == "__main__":
    arr = np.array([
        [1, 2, 3, 4],
        [3, 4, 5, 6],
        [7, 8, 9, 10],
        [1, 4, 7, 10]
    ])

    overlaps = find_overlaps(arr)
    for overlap in overlaps:
        print("Overlap between arrays {} and {}: indices {}".format(overlap[0], overlap[1], overlap[2]))

array indices:  [0, 3]
array indices:  [0, 1]
array indices:  [0, 1, 3]
array indices:  [2, 3]
array indices:  [2, 3]
Overlap between arrays 0 and 3: indices 1
Overlap between arrays 0 and 1: indices 3
Overlap between arrays 0 and 1: indices 4
Overlap between arrays 0 and 3: indices 4
Overlap between arrays 1 and 3: indices 4
Overlap between arrays 2 and 3: indices 7
Overlap between arrays 2 and 3: indices 10


In [78]:
def get_array_indices(arr):
    # Flatten the input array to work with 1D arrays
    flat_arr = np.vstack(arr)
    
    # Create an array of unique elements and their counts
    unique_elements, counts = np.unique(flat_arr, return_counts=True)
    
    # Find elements that appear in more than one array
    overlapping_elements = unique_elements[counts > 1]
    
    # Initialize a list to store the overlaps
    indices = []
    
    # Iterate through each overlapping element
    for element in overlapping_elements:
        # Find the indices of arrays that contain the element
        array_indices = [i for i, a in enumerate(arr) if np.isin(element, a)]
        indices.append(array_indices)
    
    return indices

In [80]:
get_array_indices(quads[:5000])

[[3552, 3554],
 [3365, 3366],
 [4433, 4434],
 [3880, 3881],
 [167, 168],
 [588, 1719],
 [922, 923],
 [1321],
 [713, 714],
 [4421],
 [2878, 2889],
 [2844, 2846, 2848],
 [3541, 3549],
 [4221, 4222],
 [1154, 1155, 1156, 1157],
 [186, 187, 188, 189, 190, 191, 192, 193],
 [793, 1334, 1524, 1645],
 [2306, 2307],
 [4502, 4503],
 [3541, 3549],
 [729, 1668],
 [821, 4655],
 [4969, 4970],
 [4969, 4970],
 [774, 775],
 [1116, 1122],
 [1011, 1012],
 [4382, 4383],
 [4502, 4503],
 [3788, 3789],
 [2307, 2308],
 [3537, 3538, 3539],
 [1490, 1684],
 [1125, 1291],
 [1244, 1245, 1246, 1247, 1248],
 [1242, 1243],
 [2835, 2836],
 [3044, 3963],
 [4434, 4435],
 [4451],
 [4434, 4435],
 [4426, 4427],
 [3607, 3608],
 [3605, 3606, 3607, 3608],
 [1127, 1296],
 [1423, 1424],
 [1125, 1291],
 [922, 923],
 [1125, 1291],
 [1423, 1424],
 [713, 714],
 [4011, 4063],
 [774, 775],
 [2878, 2889],
 [2878, 2889],
 [3660],
 [3500, 3502, 3503],
 [3006, 3013],
 [4382, 4383],
 [990, 1219],
 [1011, 1012],
 [412, 413],
 [1524, 1525],


In [22]:
def get_unique_elements(arr):
    # Flatten the input array to work with 1D arrays
    flat_arr = np.vstack(arr)
    
    # Create an array of unique elements and their counts
    unique_elements, counts = np.unique(flat_arr, return_counts=True)
    return(unique_elements, counts)

In [23]:
import numpy as np

def find_overlaps(arr):
    # Flatten the input array to work with 1D arrays
    flat_arr = np.vstack(arr)

    # Create a boolean mask of elements that appear more than once
    mask = np.isin(flat_arr, np.unique(flat_arr)[np.unique(flat_arr, return_counts=True)[1] > 1])
    #print("mask:", mask)
    print("mask size:", mask.size)

    # Reshape the mask back to the original shape
    mask_reshaped = mask.reshape(arr.shape)

    # Find the indices where the mask is True along the last axis
    overlaps_indices = np.argwhere(mask_reshaped)
    #print("overlaps_indices:", overlaps_indices)
    print("size: ", overlaps_indices.size)

    # Group the indices by the first index, indicating which array they belong to
    overlaps_groups = np.split(overlaps_indices[:, 1:], np.cumsum(np.unique(overlaps_indices[:, 0], return_counts=True)[1])[:-1])

    # Create a list of overlaps tuples
    overlaps = [(i, j, indices.flatten()) for i, indices_group in enumerate(overlaps_groups) for j, indices in enumerate(overlaps_groups) if i < j]

    return overlaps

# Test the function
if __name__ == "__main__":

    overlaps = find_overlaps(quads)
    print(len(overlaps))
    #print("Overlap between arrays {} and {}: indices {}".format(overlap[0], overlap[1], overlap[2]))
        

mask: [[False False False False]
 [False False False False]
 [False False False False]
 ...
 [False False False False]
 [False False False False]
 [False False False False]]
mask size: 4000
overlaps_indices: [[167   2]
 [167   3]
 [168   0]
 [168   1]
 [186   0]
 [187   0]
 [188   0]
 [189   0]
 [190   0]
 [191   0]
 [192   0]
 [193   0]
 [316   3]
 [317   0]
 [412   3]
 [413   0]
 [713   2]
 [713   3]
 [714   0]
 [714   1]
 [774   2]
 [774   3]
 [775   1]
 [775   2]
 [922   2]
 [922   3]
 [923   0]
 [923   1]]
size:  56
Overlap between arrays 0 and 1: indices [0 1]
Overlap between arrays 0 and 2: indices [0]
Overlap between arrays 0 and 3: indices [0]
Overlap between arrays 0 and 4: indices [0]
Overlap between arrays 0 and 5: indices [0]
Overlap between arrays 0 and 6: indices [0]
Overlap between arrays 0 and 7: indices [0]
Overlap between arrays 0 and 8: indices [0]
Overlap between arrays 0 and 9: indices [0]
Overlap between arrays 0 and 10: indices [3]
Overlap between arrays 0 and 1

In [26]:
flat_arr = np.vstack(quads)

# Create a boolean mask of elements that appear more than once
mask = np.isin(flat_arr, np.unique(flat_arr)[np.unique(flat_arr, return_counts=True)[1] > 1])
#print("mask:", mask)
print("mask size:", mask.size)

# Reshape the mask back to the original shape
mask_reshaped = mask.reshape(quads.shape)

# Find the indices where the mask is True along the last axis
overlaps_indices = np.argwhere(mask_reshaped)
#print("overlaps_indices:", overlaps_indices)
print("size: ", overlaps_indices.size)

# Group the indices by the first index, indicating which array they belong to
overlaps_groups = np.split(overlaps_indices[:, 1:], np.cumsum(np.unique(overlaps_indices[:, 0], return_counts=True)[1])[:-1])


mask size: 1454792
size:  2581264


In [23]:
flat_arr = np.vstack(quads)
tokens, counts = np.unique(flat_arr, return_counts=True)
multiples = counts > 1 
print(multiples)
multiple_tokens = tokens[multiples]
print(multiple_tokens)

[False False False ... False False False]
[    30     32     75 ... 472819 472823 472844]


In [29]:
tokens

array([     0,      1,      2, ..., 472900, 472901, 472902])

In [30]:
quads

array([[371438, 315167,  92886, 426566],
       [206210,  73262,  26180, 454229],
       [171706, 371431,  25901,  75214],
       ...,
       [191785, 173325, 173449,  65794],
       [ 99761, 322440, 384271, 409696],
       [ 36617, 185203,  34077, 196028]])

In [32]:
np.vstack(flat_arr)

array([[371438, 315167,  92886, 426566],
       [206210,  73262,  26180, 454229],
       [171706, 371431,  25901,  75214],
       ...,
       [191785, 173325, 173449,  65794],
       [ 99761, 322440, 384271, 409696],
       [ 36617, 185203,  34077, 196028]])

In [35]:
quad_counts = []
for arr in quads: 
    num_arr = []
    for num in arr:
        num_arr.append(counts[num])
    quad_counts.append(num_arr)

In [36]:
quad_counts

[[1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 2, 1, 1],
 [1, 1, 1, 1],
 [1, 2, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [4, 4, 1, 1],
 [4, 4, 1, 1],
 [4, 4, 1, 1],
 [4, 4, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 2, 2, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 2, 2, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1,

In [40]:
quad_count_sums = [sum(arr) for arr in quad_counts]

In [42]:
quad_count_sums

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 10,
 10,
 10,
 10,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 4,
 4,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 12,
 12,
 12,
 4,
 5,
 4,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 7,
 5,
 5,
 7,
 6,
 5,
 8,
 6,
 4,
 9,
 4,
 7,
 6,
 6,
 6,
 6,
 4,
 4,
 9,
 7,
 5,
 8,
 7,
 6,
 8,
 6,
 8,
 6,
 4,
 9,
 4,
 7,
 10,
 5,
 7,
 4,
 4,
 6,
 4,
 6,
 7,
 5,
 6,
 5,
 9,
 10,
 11,
 12,
 13,
 12,
 12,
 6,
 7,
 7,
 8,
 5,
 7,
 6,
 7,
 6,
 7,
 4,
 5,
 4,
 5,
 6,
 4,
 8,
 4,
 4,
 5,
 5,
 5,
 4,
 6,
 7,
 4,
 5,
 5,
 8,
 6,
 7,
 12,
 8,
 10,
 11,
 4,
 6,
 8,
 7,
 6,
 6,
 10,
 10,
 7,
 10,
 8,
 11,
 12,
 9,
 8,
 9,
 8,
 6,
 5,
 7,
 4,
 4,
 6,
 7,
 4,
 5,
 7,
 11,
 15,
 12,
 13,
 12,
 8,
 16,
 6,
 6,
 4,
 6,
 4,
 6,
 7,
 11,
 12,
 12,
 14,
 11,
 10,
 10,
 12,
 9,
 8,
 12,

In [46]:
for i, x in enumerate(quad_count_sums[:1000]):
    if not x==4:
        print(i,x)

30 5
32 5
46 10
47 10
48 10
49 10
56 6
60 6
69 12
70 12
71 12
73 5
75 5
92 6
93 6
103 7
104 5
105 5
106 7
107 6
108 5
109 8
110 6
112 9
114 7
115 6
116 6
117 6
118 6
121 9
122 7
123 5
124 8
125 7
126 6
127 8
128 6
129 8
130 6
132 9
134 7
135 10
136 5
137 7
140 6
142 6
143 7
144 5
145 6
146 5
147 9
148 10
149 11
150 12
151 13
152 12
153 12
154 6
155 7
156 7
157 8
158 5
159 7
160 6
161 7
162 6
163 7
165 5
167 5
168 6
170 8
173 5
174 5
175 5
177 6
178 7
180 5
181 5
182 8
183 6
184 7
185 12
186 8
187 10
188 11
190 6
191 8
192 7
193 6
194 6
195 10
196 10
197 7
198 10
199 8
200 11
201 12
202 9
203 8
204 9
205 8
206 6
207 5
208 7
211 6
212 7
214 5
215 7
216 11
217 15
218 12
219 13
220 12
221 8
222 16
223 6
224 6
226 6
228 6
229 7
230 11
231 12
232 12
233 14
234 11
235 10
236 10
237 12
238 9
239 8
240 12
241 7
242 5
243 7
244 7
245 6
246 5
247 6
248 7
249 6
250 6
251 5
252 9
253 6
254 5
255 6
256 6
257 6
258 5
259 7
260 7
261 7
262 7
263 6
264 7
265 6
266 6
267 5
268 5
270 5
271 6
272 6
273 5


In [43]:
quad_indices_sorted = sorted(range(len(quad_count_sums)), key=lambda k: quad_count_sums[k])

In [49]:
quad_indices_sorted[-100:]

[64464,
 69163,
 69410,
 69412,
 69414,
 101025,
 101027,
 101029,
 40199,
 40200,
 40202,
 40204,
 40206,
 96870,
 96873,
 96874,
 96875,
 112058,
 112059,
 112063,
 24018,
 24224,
 24273,
 24380,
 24435,
 53237,
 55936,
 55941,
 100531,
 100537,
 16908,
 16909,
 16910,
 16911,
 16912,
 16913,
 16914,
 16915,
 97349,
 97350,
 97351,
 97352,
 97353,
 112060,
 28822,
 55170,
 61476,
 90387,
 91398,
 103276,
 63558,
 63559,
 63560,
 63561,
 63562,
 63563,
 74386,
 74387,
 74388,
 74389,
 74390,
 74391,
 77000,
 77001,
 77002,
 77003,
 77004,
 77005,
 61435,
 61438,
 64459,
 64462,
 101307,
 101308,
 101309,
 101310,
 101311,
 101312,
 101313,
 92564,
 92565,
 92566,
 92567,
 92568,
 92569,
 92570,
 92571,
 86085,
 86087,
 86089,
 86091,
 86093,
 86095,
 86097,
 92037,
 92039,
 92041,
 92043,
 92045,
 92047]

In [59]:
double_num = (np.array(quad_count_sums)>4).sum()
num_to_delete = math.ceil(double_num/10)

In [61]:
to_delete = np.array(quad_indices_sorted[-num_to_delete:])

In [28]:
print(counts)

[1 1 1 ... 1 1 1]


In [24]:
len(tokens)

472903

In [25]:
len(multiple_tokens)

27054

In [26]:
mask = np.isin(flat_arr, multiple_tokens)

In [27]:
mask

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       ...,
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

In [65]:
array_sums = np.array([ar.sum() for ar in mask])


In [126]:
smaller_four = array_sums < 4
np.argwhere(smaller_four).flatten()

array([     0,      1,      2, ..., 169319, 169320, 169321], dtype=int64)

In [95]:
equals_four = array_sums==4
equals_four_indices = np.argwhere(equals_four).flatten()

In [127]:
import random 
equals_four_indices_sample = random.choices(equals_four_indices, k = int(equals_four_indices.size/2))

In [128]:
len(equals_four_indices)

320498

In [129]:
len(equals_four_indices_sample)

160249

In [130]:
to_keep = smaller_four.copy()
to_keep[equals_four_indices_sample] = True

IndexError: index 230994 is out of bounds for axis 0 with size 169322

In [103]:
smaller_four.sum()

43200

In [102]:
to_keep.sum()

169322

In [104]:
quads_to_keep = quads[to_keep]

In [107]:
new_elements, new_counts = np.unique(quads_to_keep, return_counts=True)
new_elements.size

565890

In [118]:
x = new_counts > 1 

In [122]:
x

array([False, False, False, ..., False, False, False])

In [119]:
x.sum()

106195

In [123]:
mask = np.isin(quads_to_keep, new_elements[new_counts > 1])

In [124]:
mask

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       ...,
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

In [125]:
array_sums = np.array([ar.sum() for ar in mask])
still_equals_four = array_sums==4
still_equals_four.sum()

50632

In [113]:
equals_one = array_sums==1
equals_one.sum()

2

In [114]:
(np.unique(quads_to_keep, return_counts=True)[1] > 1).sum()

106195

In [75]:
smaller_four.sum()

43200

In [71]:
len(smaller_four)

363698

In [77]:
quads_smaller_four = quads[smaller_four]


In [83]:
(np.unique(quads_smaller_four, return_counts=True)[1] > 1).sum()

1111

In [85]:
new_elements, new_counts = np.unique(quads_smaller_four, return_counts=True)
new_elements.size

171519

In [35]:
quads

array([[ 65332,  61419,  87689, 696034],
       [752366, 143559, 583505, 275851],
       [557935, 672033, 411953,  37402],
       ...,
       [496711, 254275, 593174, 531678],
       [708638, 519395, 620721, 708518],
       [303874,  36521, 444815, 400026]], dtype=int64)

In [36]:
print(quads * mask_reshaped)

[[     0      0      0      0]
 [     0      0      0      0]
 [     0      0      0      0]
 ...
 [496711 254275 593174 531678]
 [708638 519395 620721 708518]
 [303874  36521 444815 400026]]


In [28]:
len(overlaps_groups)

324748

In [30]:
print(overlaps_groups[:10])

[array([[2],
       [3]], dtype=int64), array([[0],
       [1]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64), array([[0]], dtype=int64)]
